In [3]:
import GloVepreprocessing
import model


import numpy as np
import logging

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Lambda 
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Reshape, LSTM, Embedding, TimeDistributed
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [2]:
preprocessor = GloVepreprocessing.GloVepreprocessor()

In [ ]:
print(preprocessor.tokenizer.get_config().keys())
print(len(preprocessor.tokenizer.get_config()['word_counts']))
print(len(preprocessor.tokenizer.get_config()['word_index']))
print(len(preprocessor.tokenizer.get_config()['index_word']))

In [ ]:
dict1={"la":0, "le":1, "li":2}
dict2={"lustucru":0, "lu":1, "lou":2, "bli":3}
dict2.pop("lustucru")
n = len(dict1)
newdict = dict1
for key, value in dict2.items():
    newdict[key]=int(value)+n-1
print(newdict)

In [ ]:
sentences = ["Il etait une fois", "Y'en a marre de cette chanson la!"]

embedded_list = preprocessor.GloVe_embed(sentences, preprocessor.weights, isInputSentence=False)
print(embedded_list)
print(np.reshape(embedded_list, (2,35,50)))

In [ ]:
PAD_IDX = 0
CLS_IDX = 1
SEP_IDX = 2
UNKNOWN_IDX = 3
preword2idx = { '[PAD]': PAD_IDX, '[CLS]': CLS_IDX, '[SEP]': SEP_IDX, '[UNK]': UNKNOWN_IDX}
preidx2word = { PAD_IDX :'[PAD]' , CLS_IDX :'[CLS]' , SEP_IDX :'[SEP]' , UNKNOWN_IDX :'[UNK]'}        

postword2idx = preprocessor.tokenizer.word_index
postidx2word = preprocessor.tokenizer.index_word

n = len(preword2idx)

word2idx = preword2idx
idx2word = preidx2word
for key, value in postword2idx.items():
    word2idx[key]=int(value)+n-1
for key, value in postidx2word.items():
    idx2word[int(key)+n-1]=value
print(idx2word)
#print(word2idx)

In [ ]:
def concat_dict

In [ ]:
my_batch = preprocessor.batch_generator("train",0, 20)
print(len(my_batch[1]["caption_output"]))

In [ ]:
#preprocessor = GloVepreprocessing.GloVepreprocessor()

X, Y  = preprocessor.batch_generator("train", 0, 100)


mymodel = model.TrainShowAndTell(preprocessor.MAX_SEQUENCE_LENGTH, preprocessor.VOCAB_SIZE, preprocessor.EMBEDDING_SIZE, 60, preprocessor.weights)

mymodel.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.01), metrics=['accuracy'])

a0 = np.zeros((X["image_input"].shape[0], 60))
c0 = np.zeros((X["image_input"].shape[0], 60))

mymodel.fit([X["image_input"],X["caption_input"]
             , a0, c0], Y["caption_output"], batch_size=10, epochs=9, verbose=2)

In [3]:

def one_hot_encode(captions, max_caption_length, n_classes):
    hot_encoding = np.zeros((len(captions), max_caption_length, n_classes))
    for i, caption in enumerate(captions):
        for j, idx in enumerate(caption):
            hot_encoding[i, j, idx] = 1.0
        for k in range(j+1, max_caption_length):
            hot_encoding[i, k, preprocessor.word2idx["[PAD]"]] = 1.0

    return hot_encoding

In [15]:
captions = ["once upon a time","in a far away cabin"]
list_of_captions_ids = preprocessor.get_sentences_ids(preprocessor.preprocess_sentences(captions), preprocessor.tokenizer, isInputSentence=False)
print(len(list_of_captions_ids))
print(list(list_of_captions_ids))
one_hot_encode(list_of_captions_ids,35,preprocessor.VOCAB_SIZE)


2
[array([[ 973, 1977,    5,  114,    2,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int32), array([[   7,    5, 1167,  439, 1753,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int32)]


array([[[1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
  list_of_captions_ids = self.get_sentences_ids(self.preprocess_sentences(list_of_captions), self.tokenizer, isInputSentence)
    
        for caption_ids in list_of_captions_ids:
            for id in caption_ids:
                caption_embedding = []
                caption_embedding.append(self.weights[id])
            embeddings.extend(caption_embedding)
    
        return embeddings
    

In [ ]:
def generate_caption(model, tokenizer, image, max_length):
	# Seed the generation process
	in_text = 'startseq'
	# Iterate over the whole length of the sequence
	for _ in range(max_length):
		# Integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# Pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# Predict next word
		# The model will output a prediction, which will be a probability distribution over all words in the vocabulary.
		yhat = model.predict([image,sequence], verbose=0)
		# The output vector representins a probability distribution where maximum probability is the predicted word position
		# Take output class with maximum probability and convert to integer
		yhat = np.argmax(yhat)
		# Map integer back to word
		word = int_to_word(yhat, tokenizer)
		# Stop if we cannot map the word
		if word is None:
			break
		# Append as input for generating the next word
		in_text += ' ' + word
		# Stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
